# Importing libraries

In [1]:
import os
import sys
import glob
import nilearn
from nilearn import image
sys.path.append("..")

import json
from tqdm import tqdm
from easydict import EasyDict as edict

from utils.interpolation import *

/usr/local/lib/python3.6/dist-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()


# Specifying trained model

In [2]:
EXP_NAME = '2022-03-18_experiment_a7431019-46fe-478c-83a5-ef394784f7fa'

with open(f'experiments/{EXP_NAME}/config.json') as json_file:
    config = json.load(json_file)
    
config = edict(config)

os.makedirs(f'experiments/{EXP_NAME}/interpolation', exist_ok=True)

# Interpolation

In [ ]:
for subject in tqdm(config.subjects):
    
    with open(f'experiments/{config.name_of_experiment}/predictions/{subject}.json') as json_file:
        net_output = json.load(json_file)
    
    coordinates, predictions = net_output['coordinates'], net_output['predictions']
    result = points2mri(coordinates, predictions)
    
    original_brain = nib.load(f"{config.path_to_data}/{config.brains_modality}_brains/{subject}.nii.gz")
    brain_mask = nib.load(f'radiologist/preprocessed_data/new_pipeline/sub-{subject}/sub-{subject}_acq-T1Mprage_space-MNI152NLint2_seq-T1w_brain.nii_mask.nii.gz')
    brain_mask = nilearn.image.resample_to_img(brain_mask, original_brain, interpolation='nearest')
    
    pred = result * brain_mask.get_fdata()
    interpolation = nib.Nifti1Image(pred, original_brain.affine)
    nib.save(interpolation, f"experiments/{config.name_of_experiment}/interpolation/{subject}.nii.gz")

 95%|█████████▌| 125/131 [3:41:39<10:38, 106.40s/it] 